In [ ]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}

In [ ]:
%run __init__.ipynb

In [ ]:
BHC = bob.HealthCheck()
BHC_KEY = "99c0c879-0fe1-45c5-b5a6-95f0c2b1b203"
if USE_HEALTHCHECKS:
    STEP_1 = BHC.start(BHC_KEY)
    print(STEP_1)

In [ ]:
# --- Covid-19 France data updated everyday at 19:00PM
%run "covid-19_france.ipynb"

In [ ]:
# --- Load data source
START_TIME = time.time()
DB_ALL_FR = get_datasource('FRANCE_DB_ALL')
DB_TREND_FR = get_datasource('FRANCE_DB_TREND')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")

In [ ]:
# -- Covid-19 - DATA.GOUV.FR TENDANCES
START_TIME = time.time()


def domain_105(dataf, domain_num):
    """
    doc string
    """
    # -- Create Datamodel
    domain = dataf.copy()
    domain = pd.DataFrame({'ENTITY': 'France',
                           'SCENARIO': domain['SCENARIO'],
                           'GROUPS': domain['STATUS_NAME'],
                           'GROUPS_ORDER': domain['STATUS_ORDER'],
                           'DATE_ORDER': domain['DATE_ORDER'],
                           'BOTTOM_FILTER': 'Depuis le début',
                           'UPPER_FILTER_M': domain['METRIC'],
                           'UPPER_FILTER_R': domain['LABEL'],
                           'LABEL_GROUPS': domain['LABEL_GROUPS'],
                           'VALUE': domain['VALUE'],
                           'UNIT_VALUE': domain['UNIT'],
                           'PRECISION': domain['PRECISION']})

    # -- Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE',
               'UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARV',
               'UPPER_FILTER_M'] = 'VARIATION EN NB'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARP',
               'UPPER_FILTER_M'] = 'VARIATION EN %'

    # -- Create new date scenario in BOTTOM_FILTER
    df1 = get_lastdays(domain, 15, "15 derniers jours", 'BOTTOM_FILTER')
    domain = pd.concat([domain, df1], axis=0)

    # -- Filter upperfilter value to get France and detailed by region
    domain_dep = domain[domain['LABEL_GROUPS'] != 'France']
    domain_fra = domain[domain['UPPER_FILTER_R'] == 'France']
    domain = pd.concat([domain_dep, domain_fra], axis=0)

    # -- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE'] = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=';')
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)

    # -- Create upper filter domain
    upper_filter = (domain[['UPPER_FILTER_R']].
                    drop_duplicates().
                    sort_values(by='UPPER_FILTER_R', ascending=True).
                    reset_index(drop=True))
    upper_filter['ORDER'] = upper_filter.index + 1
    upper_filter.loc[upper_filter['UPPER_FILTER_R'] == 'France', 'ORDER'] = 0
    upper_filter = upper_filter.sort_values(by='ORDER', ascending=True)
    upper_filter.to_csv(os.path.join(OUTPUT_FOLDER,
                                     '105_UPPER_FILTER.csv'), sep=';')
    if USE_MONGO:
        bob.mongo.save_df(upper_filter, '105_UPPER_FILTER', DB_APP, True)
    return domain


DOMAIN105 = domain_105(DB_TREND_FR, '105')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
DOMAIN105

In [ ]:
# -- Covid-19 - DATA.GOUV.FR CLASSEMENT
START_TIME = time.time()


def domain_107(dataf, domain_num):
    """
    doc string
    """
    domain = dataf.copy()
    domain = pd.DataFrame({'ENTITY': 'France',
                           'SCENARIO': domain['SCENARIO'],
                           'DATE_ORDER': domain['DATE_ORDER'],
                           'CHILD': domain['LABEL'],
                           'PARENT': domain['LABEL_GROUPS'],
                           'UPPER_FILTER_M': domain['STATUS_NAME'],
                           'ORDER': domain['STATUS_ORDER'],
                           'VALUE': domain['VALUE'],
                           'VAR': domain['VARV'],
                           'VARP': domain['VARP'],
                           'UNIT_VARP': ' %',
                           'PRECISION_VALUE': ',.0f',
                           'PRECISION_VAR': '+,.0f',
                           'PRECISION_VARP': '+,.2f'})

    domain.loc[domain['PARENT'] == 'France', 'PARENT'] = 'root'

    # -- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE'] = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(os.path.join(OUTPUT_FOLDER, domain_num + '.csv'), sep=';')
    if USE_MONGO:
        bob.mongo.save_df(domain, domain_num, DB_APP, True)
    return domain


DOMAIN107 = domain_107(DB_ALL_FR, '107')
print("Script execution completed at "
      f"{datetime.now().strftime('%d/%m/%Y %H:%M:%S')}. "
      f"Time: --- {time.time() - START_TIME} secnds ---")
DOMAIN107

In [ ]:
if USE_HEALTHCHECKS:
    STEP_2 = BHC.done(BHC_KEY)
    print(STEP_2)